In [1]:
import os, sys
sys.path.append(os.path.join("../.."))

import privugger.transformer.datastructures as pvds
from privugger.transformer.discrete import *
from privugger.transformer.continuous import *
from privugger.transformer.program_output import *

from privugger.transformer.method import *
from privugger.measures.mutual_information import *

import arviz as az


import pymc3 as pm

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
#####################
# PATH TO PROGRAM ##
###################
program = "program_to_analyze.py"

def alpha(age, name, alice_name):
    return (age.sum()) / (age.size)

In [3]:

##################
## AGE EXAMPLE ##
#################

N    = 10

# Specify distributions
age  = Normal(mu=55.2, std=3.5, num_elements=N)
name = DiscreteUniform(0, 5, num_elements=N)
    
age_alice  = Uniform(lower=0, upper=100, num_elements=1)
name_alice = DiscreteUniform(0, 5, num_elements=1) 


dist_stack_1 = Normal(mu=0, std=15, num_elements=N)
dist_stack_2 = Normal(mu=1, std=5, num_elements=N)
dist_stack_3 = Normal(mu=2, std=1, num_elements=N)


#This is an example of how to concatenate
con  = concatenate(age_alice, age)

#This is not used in the program, just an example to illustrate how to use
stacked = stack([dist_stack_1, dist_stack_2, dist_stack_3], axis=0)

# Create dataset. Refer to "age_alice" as "age1" in the trace and "age" as "age2" in the trace. This is the general naming convention. 
ds   = pvds.Dataset(input_specs = [con, stacked, name, name_alice],
                  var_names   = ["age", "stacked", "name", "name_alice"])

# Call infer and specify program output
# For now output type can be: Int, Float, List(Float), List(Int)
trace = infer(ds, Float,  alpha, cores=2, draws=1000)

import theano
import theano.tensor as tt
import numpy as np


def method(age, name, alice_name):

    @theano.compile.ops.as_op(itypes=[tt.dvector, tt.lvector, tt.
        TensorType('int64', (True,))], otypes=[tt.dscalar])
    def alpha(age, name, alice_name):
        return np.array(age.sum() / age.size)
    return alpha(age, name, alice_name)

[age, name, name_alice]


Multiprocess sampling (2 chains in 2 jobs)
CompoundStep
>NUTS: [age]
>CompoundStep
>>Metropolis: [name_alice]
>>Metropolis: [name]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 3 seconds.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


In [19]:
#age = trace['b, a']
#age
#out = trace['output']
#trace[0]
#mi_sklearn(age, out, n_neigh=5)

{'age1_interval__': array([-1.77085666]),
 'age2': array([58.03911816, 54.4445012 , 55.41451912, 60.42400327, 58.04198223,
        58.71457342, 51.54331872, 54.08231318, 58.98070821, 63.80370524]),
 'name': array([2, 3, 2, 5, 1, 3, 4, 3, 5, 0]),
 'name_alice': array([5]),
 'age1': array([14.54358271]),
 'output': 53.457484130756036}

In [4]:

N = 10

#alice_name      = Constant(0)
#alice_age       = Constant(25)
#alice_zip       = Constant(50)
#alice_gender    = Constant(1) 
#alice_diagnosis = Constant(1) 
    
    
names = DiscreteUniform(0, 5, num_elements=N)
ages  = Normal(mu=55.2, std=3.5, num_elements=N)

zips = DiscreteUniform(0, 100, num_elements=N)

genders = Bernoulli(p=.5, num_elements=N)   
diagnoses = Bernoulli(p=.2,num_elements=N)

ds = pvds.Dataset(input_specs = [names, ages, zips, genders, diagnoses], 
                  var_names=["names", "ages", "zips", "genders", "diagnoses"])


trace = infer(ds, stack=True)

[names, ages, zips, genders, diagnoses]
names
ages
zips
genders
diagnoses


Multiprocess sampling (2 chains in 2 jobs)
CompoundStep
>CompoundStep
>>Metropolis: [zips]
>>Metropolis: [names]
>NUTS: [ages]
>BinaryGibbsMetropolis: [genders, diagnoses]


Sampling 2 chains for 1_000 tune and 500 draw iterations (2_000 + 1_000 draws total) took 3 seconds.
The rhat statistic is larger than 1.2 for some parameters.
The estimated number of effective samples is smaller than 200 for some parameters.
